# dots.ocr: Обработка PDF с дообученной моделью

In [1]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers accelerate qwen-vl-utils
!pip install -q pillow tqdm pymupdf pandas markdown pydantic openai requests
!pip install -q typing-extensions>=4.12.2
print("Зависимости установлены")

Зависимости установлены


In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Память: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"Занято: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
    print(f"Свободно: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved(0)) / 1e9:.2f} GB")
else:
    print("WARNING: GPU не обнаружен, используется CPU")

GPU: NVIDIA A100-SXM4-80GB
Память: 85.1 GB
Занято: 0.00 GB
Свободно: 85.10 GB


## Конфигурация

In [3]:
MODEL_PATH = "/home/jovyan/kurkin/geo_bench_zagorulko/OCR/DotsOCR_finetuned"
PDF_PATH = "/home/jovyan/kurkin/geo_bench_zagorulko/OCR/data/TAR_ie_2023_12_06.pdf"
OUT_DIR = "/home/jovyan/kurkin/geo_bench_zagorulko/OCR/outputs"
DOTS_OCR_PATH = "/home/jovyan/kurkin/geo_bench_zagorulko/OCR/dots_ocr"

# Параметры
DPI = 200
PROMPT_MODE = "prompt_layout_all_en"  
MAX_NEW_TOKENS = 24000
USE_FAST_INFERENCE = True

USE_VLLM = True
VLLM_SERVER_IP = "localhost"
VLLM_SERVER_PORT = 8000
VLLM_PROTOCOL = "http"
VLLM_MODEL_NAME = MODEL_PATH

from pathlib import Path # Проверка путей
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

errors = []
for name, path in [("PDF", PDF_PATH), ("Модель", MODEL_PATH), ("dots.ocr", DOTS_OCR_PATH)]:
    exists = Path(path).exists()
    print(f"{name}: {'OK' if exists else 'ERROR'}")
    if not exists:
        errors.append(f"{name}: {path}")

if errors:
    raise FileNotFoundError(f"Не найдены пути: {errors}")

print(f"\nРежим: {'vLLM' if USE_VLLM else 'HF Transformers'}")
print(f"Модель: Дообученная (объединенная)")

PDF: OK
Модель: OK
dots.ocr: OK

Режим: vLLM
Модель: Дообученная (объединенная)


## Подключение dots.ocr

In [4]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path(DOTS_OCR_PATH)))

try:
    import dots_ocr
except ImportError:
    import subprocess
    subprocess.run(["pip", "install", "-e", str(DOTS_OCR_PATH)], check=False)

try:
    from dots_ocr import DotsOCRParser
except ImportError:
    from dots_ocr.parser import DotsOCRParser

print("dots.ocr подключен")

dots.ocr подключен


In [5]:
if USE_VLLM:
    try:
        import requests
        test_url = f"{VLLM_PROTOCOL}://{VLLM_SERVER_IP}:{VLLM_SERVER_PORT}/health"
        response = requests.get(test_url, timeout=2)
        if response.status_code == 200:
            print("vLLM сервер доступен")
        else:
            print(f"WARNING: vLLM сервер статус {response.status_code}")
    except Exception as e:
        print(f"WARNING: vLLM сервер недоступен: {e}")
    
    parser = DotsOCRParser(
        protocol=VLLM_PROTOCOL, ip=VLLM_SERVER_IP, port=VLLM_SERVER_PORT,
        model_name=VLLM_MODEL_NAME, use_hf=False,
        dpi=DPI, output_dir=OUT_DIR, num_thread=4
    )
    print("Парсер с vLLM готов")
else:
    parser = DotsOCRParser(use_hf=True, dpi=DPI, output_dir=OUT_DIR)
    # Переопределяем путь к модели для HF режима
    parser.model_path = MODEL_PATH
    print("Парсер с HF Transformers готов")

vLLM сервер доступен
use vllm model, num_thread will be set to 4
Парсер с vLLM готов


## Обработка PDF

In [6]:
result = parser.parse_file(PDF_PATH, prompt_mode=PROMPT_MODE, fitz_preprocess=False)
print(f"\nГотово! Результаты в {OUT_DIR}")

loading pdf: /home/jovyan/kurkin/geo_bench_zagorulko/OCR/data/TAR_ie_2023_12_06.pdf
Parsing PDF with 42 pages using 4 threads...


Processing PDF pages: 100%|██████████| 42/42 [01:48<00:00,  2.58s/it]

Parsing finished, results saving to /home/jovyan/kurkin/geo_bench_zagorulko/OCR/outputs/TAR_ie_2023_12_06

Готово! Результаты в /home/jovyan/kurkin/geo_bench_zagorulko/OCR/outputs


## Объединение Markdown и восстановление PDF

In [7]:
from pathlib import Path
import re
import json
import markdown as md
from PIL import Image

pdf_filename = Path(PDF_PATH).stem
save_dir = Path(OUT_DIR) / pdf_filename

json_files = list(save_dir.glob("*_page_*.json")) if save_dir.exists() else []
md_files = list(save_dir.glob("*_page_*.md")) if save_dir.exists() else []

def extract_page_num(filename):
    match = re.search(r'_page_(\d+)', filename.stem)
    return int(match.group(1)) if match else 999

json_files.sort(key=extract_page_num)
valid_md_files = [f for f in md_files if not f.name.endswith("_nohf.md")]
valid_md_files.sort(key=extract_page_num)

if json_files or valid_md_files:
    combined_md = []
    combined_md.append(f"# {pdf_filename}\n\n")
    combined_md.append(f"*Объединенный документ из {max(len(json_files), len(valid_md_files))} страниц*\n\n")
    combined_md.append("---\n\n")
    
    if json_files:
        print("Используем JSON файлы для восстановления (более точный порядок элементов)")
        try:
            from dots_ocr.utils.format_transformer import layoutjson2md
            
            for json_file in json_files:
                page_num = extract_page_num(json_file)
                
                # Загружаем JSON
                with open(json_file, 'r', encoding='utf-8') as f:
                    cells = json.load(f)
                
                # Загружаем соответствующее изображение страницы
                img_path = json_file.with_suffix('.jpg')
                if not img_path.exists():
                    img_path = json_file.parent / f"{json_file.stem.replace('.json', '')}.jpg"
                
                if img_path.exists():
                    origin_image = Image.open(img_path)
                    # Генерируем Markdown из JSON с правильным порядком
                    md_content = layoutjson2md(origin_image, cells, text_key='text', no_page_hf=True)
                else:
                    # Если нет изображения, используем готовый MD файл
                    md_file = json_file.with_suffix('.md')
                    if md_file.exists():
                        md_content = md_file.read_text(encoding='utf-8')
                    else:
                        # Просто текст из JSON
                        md_content = "\n\n".join([cell.get('text', '') for cell in cells if cell.get('text')])
                
                combined_md.append(f"## Страница {page_num}\n\n")
                combined_md.append(md_content)
                combined_md.append("\n\n---\n\n")
        except ImportError:
            print("Не удалось импортировать layoutjson2md, используем MD файлы")
            json_files = []
    
    if not json_files and valid_md_files:
        print("Используем MD файлы для восстановления")
        for md_file in valid_md_files:
            page_num = extract_page_num(md_file)
            content = md_file.read_text(encoding="utf-8")
            combined_md.append(f"## Страница {page_num}\n\n")
            combined_md.append(content)
            combined_md.append("\n\n---\n\n")
    
    combined_md_text = "".join(combined_md)
    combined_md_path = Path(OUT_DIR) / f"{pdf_filename}_combined.md"
    combined_md_path.write_text(combined_md_text, encoding="utf-8")
    print(f"Объединено {max(len(json_files), len(valid_md_files))} страниц")
    print(f"Markdown сохранен: {combined_md_path}")
    
    html_content = md.markdown(combined_md_text, extensions=["tables", "fenced_code", "codehilite"])
    html_with_style = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }}
        table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
        th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        th {{ background-color: #f2f2f2; }}
        code {{ background-color: #f4f4f4; padding: 2px 4px; border-radius: 3px; }}
        pre {{ background-color: #f4f4f4; padding: 10px; border-radius: 5px; overflow-x: auto; }}
        img {{ max-width: 100%; height: auto; }}
    </style>
</head>
<body>
{html_content}
</body>
</html>"""
    
    html_path = Path(OUT_DIR) / f"{pdf_filename}_combined.html"
    html_path.write_text(html_with_style, encoding="utf-8")
    print(f"HTML сохранен: {html_path}")
    
    try:
        import weasyprint
        pdf_path = Path(OUT_DIR) / f"{pdf_filename}_reconstructed.pdf"
        weasyprint.HTML(string=html_with_style).write_pdf(pdf_path)
        print(f"PDF восстановлен: {pdf_path}")
    except ImportError:
        print("weasyprint не установлен, пропускаем создание PDF")
        print("Установите: pip install weasyprint")
        print("Или откройте HTML файл в браузере для проверки")
    except Exception as e:
        print(f"Ошибка при создании PDF: {e}")
        print("Откройте HTML файл в браузере для проверки")
else:
    print("JSON и Markdown файлы не найдены")

Используем JSON файлы для восстановления (более точный порядок элементов)
Объединено 42 страниц
Markdown сохранен: /home/jovyan/kurkin/geo_bench_zagorulko/OCR/outputs/TAR_ie_2023_12_06_combined.md
HTML сохранен: /home/jovyan/kurkin/geo_bench_zagorulko/OCR/outputs/TAR_ie_2023_12_06_combined.html
PDF восстановлен: /home/jovyan/kurkin/geo_bench_zagorulko/OCR/outputs/TAR_ie_2023_12_06_reconstructed.pdf
